In [1]:
%pip install langchain_community langchain_text_splitters langchain_openai langchain_chroma gradio python-dotenv pypdf

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 1.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 7.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 10.4 MB/s eta 0:00:00
  Created wheel for pypika: filename=pypika-0.48.9-py2.py3-none-any.whl size=53770 sha256=16ffd652bbb6170e3637efb807bcde97c9ea57981624947379d94086b7e8fa5b
  Stored in directory: /Users/avineetsharma/Library/Caches/pip/wheels/f7/02/64/d541eac67ec459309d1fb19e727f58ecf7ffb4a8bf42d4cfe5
Successfully built pypika
  Attempting uninstall: importlib-metadata
    Found existing installation: importlib_metadata 8.6.1
    Uninstalling importlib_metadata-8.6.1:
   

In [ ]:
# from google.colab import drive

# # Mount Google Drive
# drive.mount('/content/drive')

# # Define the Google Drive path where your documents are stored
# google_drive_path = '/content/drive/MyDrive/local_rag/data'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
DATA_PATH = r"data"
CHROMA_PATH = r"chroma_db"

In [3]:
import os

# Get a list of all files in the specified folder
file_list = os.listdir(DATA_PATH)

# Filter the list to only include files with .pdf or .docx extensions
pdf_files = [f for f in file_list if f.endswith('.pdf')]
docx_files = [f for f in file_list if f.endswith('.docx')]

# Example: Print the list of PDF and DOCX files in the folder
print("PDF Files:", pdf_files)
print("DOCX Files:", docx_files)

PDF Files: ['AttentionIsAllyouNeed.pdf']
DOCX Files: []


In [4]:
import os
current_directory = os.getcwd()
print(current_directory)

/Users/avineetsharma/Project/Columbia/LocalDocuChat


In [5]:
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from uuid import uuid4
from langchain.embeddings import SentenceTransformerEmbeddings

# import the .env file
from dotenv import load_dotenv
load_dotenv()

# Load the embedding model
embeddings_model = SentenceTransformerEmbeddings(model_name="all-mpnet-base-v2")

# Use the model with LangChain as before
vector_store = Chroma(
    collection_name="example_collection",
    embedding_function=embeddings_model,
    persist_directory=CHROMA_PATH,
)

# loading the PDF document
raw_documents = []
for filename in os.listdir(DATA_PATH):
    if filename.endswith(".pdf"):
        filepath = os.path.join(DATA_PATH, filename)
        loader = PyPDFLoader(filepath)
        raw_documents.extend(loader.load())

# splitting the document
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=300,
    chunk_overlap=100,
    length_function=len,
    is_separator_regex=False,
)

# creating the chunks
chunks = text_splitter.split_documents(raw_documents)

# creating unique ID's
uuids = [str(uuid4()) for _ in range(len(chunks))]

# adding chunks to vector store
vector_store.add_documents(documents=chunks, ids=uuids)




/var/folders/5p/y9vfdxz132b7tj62wggjh0_m0000gn/T/ipykernel_87492/596291422.py:13: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings_model = SentenceTransformerEmbeddings(model_name="all-mpnet-base-v2")
/opt/anaconda3/envs/dev/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


['b7c7e014-3538-4212-b293-00795f9ce9a4',
 '4d5ca9af-2c5b-4222-91d8-303267944358',
 'b8393fc8-906a-43e5-a060-15514ef42414',
 'a9310c6b-894e-401d-bd7e-0a2ede4b69be',
 'bb10bb36-5e9c-4eb8-bddf-891cf803efc7',
 '4e97c03c-7c92-4d54-90e2-5489bd8a4f19',
 'dd39df12-dbac-4643-a1ee-d559e16cba55',
 '6f72ed2b-1225-4d52-8122-2c5990452fe6',
 '622b1b0f-3307-4005-89c7-1ade4966935d',
 '3ef9f3de-30de-4950-af32-c76f6b59eea2',
 'cb4de36e-bd41-42d0-8c69-0133c4b2e9a3',
 'c13c693f-b35d-4cad-8575-465c7fe16b8c',
 'f45660cd-55c6-45f1-b5b9-da1537123505',
 '4cb9a244-5061-4931-9985-98282563eaca',
 'a508f710-c135-4eff-b7af-f4cf1f35d081',
 'e30d3be9-c332-4556-8f91-f14afb4c8eda',
 '4b415af1-bb1c-47f6-92be-f2382540b7e3',
 '563e09d6-409f-4060-be0c-dfd2a558b493',
 'ba3adfbc-c396-4aea-846b-1840f2f5c497',
 '998aea54-379b-4926-968a-88b2fc0932c9',
 'eae087c4-b48e-4b70-9c38-63019fa696aa',
 'b4acee78-1d18-4b13-a2fd-222e45350a82',
 '3f3def71-e509-4ee5-9933-f5aae525f499',
 'f7b89bd7-9d5d-47b8-b445-8e206682037c',
 '4e07e7d1-976a-

In [7]:
num_results = 5

retriever = vector_store.as_retriever(search_kwargs={'k': num_results})

docs = retriever.invoke("What is attention")
breakpoint()
print(docs)

[Document(id='f1d888dd-e9a0-4199-942a-44c7708bb68b', metadata={'author': '', 'creationdate': '2024-04-10T21:11:43+00:00', 'creator': 'LaTeX with hyperref', 'keywords': '', 'moddate': '2024-04-10T21:11:43+00:00', 'page': 2, 'page_label': '3', 'producer': 'pdfTeX-1.40.25', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5', 'source': 'data/AttentionIsAllyouNeed.pdf', 'subject': '', 'title': '', 'total_pages': 15, 'trapped': '/False'}, page_content='3.2 Attention\nAn attention function can be described as mapping a query and a set of key-value pairs to an output,\nwhere the query, keys, values, and output are all vectors. The output is computed as a weighted sum\n3'), Document(id='e385e345-69fe-470b-b471-ee889bb3cf45', metadata={'author': '', 'creationdate': '2024-04-10T21:11:43+00:00', 'creator': 'LaTeX with hyperref', 'keywords': '', 'moddate': '2024-04-10T21:11:43+00:00', 'page': 6, 'page_label': '7', 'producer': 'pdfTeX-1.40.25'